# PlayHQ Fixture Scraping

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ssardina/tapp-fixture/blob/main/playhq_scrape.ipynb)

This system allows to scrape game fixtures from [PlayHQ](http://playhq.com/) via its Public [API](https://support.playhq.com/hc/en-au/sections/4405422358297-PlayHQ-APIs). It will produce a CSV file ready to be uploaded as Schedule in [TeamApp](https://brunswickmagicbasketball.teamapp.com/).

The *Public* APIs only require a header parameters to get a successful response, which includes the following components:

- `x-api-key` (also referred to as the Client ID) will be provided by PlayHQ when you request access to the public API via their [support page](https://support.playhq.com/hc/en-au) or email support@playhqsupport.zendesk.com. This key can be stored in a file `x_api_key.txt` or it will be asked interactively by the notebook otherwise. In many cases, the feature to create new API credentials is disabled for a user and can only be actioned by a Super Administrator role within the Play HQ portal.
- `x-phq-tenant` usually refers to the sport/association - in this case '`bv`'.


Detailed reference documentation for PlayHQ API can be found [here](https://docs.playhq.com/tech).

**Contact:** Sebastian Sardina (sssardina@gmail.com)

In [1]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import re
import os
import calendar, datetime
import dtale

import utils
import playhq as phq

## 1. Configuration and set-up

We first configure and set-up the application. This means reading configuration variables from a config file and setting the game day.

So, first of all, specify the following information:

1. Configuration file for the club and season.
2. Game dates interval to scrape.

In [2]:
# Change this to import your club's own configuration
# from config_bmc_w23 import *
from config_bmc_s23 import *
# from config_cba_24 import *

# Set the game date interval scrape
# GAME_DATE_START = datetime.date.today() # by default, any game after today
# GAME_DATE_START = utils.next_day(calendar.SATURDAY)   # start from next game day (used for Domestic)
GAME_DATE_START = datetime.date(2024, 3, 12) # start on a specific day - (used for Rep - REP ROUND 5+)

WEEKS = 10   # how many weeks after date start we want to scrape (use 1 for just next game)
GAME_DATE_END = GAME_DATE_START + datetime.timedelta(days=WEEKS*7)


###############################################################
# DO NOT CHANGE FROM HERE
###############################################################

# Get nice game date format: Saturday August 06, 2022
GAME_DATE_START_TIMESTAMP = pd.to_datetime(GAME_DATE_START).tz_localize(TIMEZONE)
GAME_DATE_END_TIMESTAMP = pd.to_datetime(GAME_DATE_END).tz_localize(TIMEZONE)

GAME_DATE_START_NAME = utils.pretty_date(GAME_DATE_START_TIMESTAMP)
GAME_DATE_END_NAME = utils.pretty_date(GAME_DATE_END_TIMESTAMP)

# Create phq_club object
phq_club = phq.PlayHQ(CLUB_NAME, ORG_ID, X_API_KEY, X_TENANT, TIMEZONE, tapp_team_name, tapp_game_name)
if SEASON_ID is None:
    raise SystemExit("ERROR! Please specify either SEASON_ID.")
SEASON_NAME = phq_club.get_season_name(SEASON_ID)

print(f"Club name: {CLUB_NAME} (org. id: {ORG_ID})")
print(f"Season: {SEASON_NAME} (season id: {SEASON_ID})")
print("X-tenant:", X_TENANT, "x-api-key:", X_API_KEY)
print("Output path:", OUTPUT_PATH)
if not os.path.exists(OUTPUT_PATH):
    raise SystemExit("ERROR! Output path {OUTPUT_PATH} is missing! Please create or link that path correctly to save data.")

print("Timezone:", TIMEZONE)
print(f"Game dates: {GAME_DATE_START_NAME} - {GAME_DATE_END_NAME}")
print("PlayHQ Club fixture:", PLAYHQ_SEASON_URL)

Club name: Brunswick Magic Basketball Club (org. id: 8c4d5431-eaa5-4644-82ac-992abe224b88)
Season: Junior Domestic (season id: 2485b3f7-55de-4b88-a3b6-527c36717fee)
X-tenant: bv x-api-key: f5d33c76-f858-49fa-8330-8e0e396219cd
Output path: Brunswick_Magics/2023.02.Summer-23_24/fixture/
Timezone: Australia/Melbourne
Game dates: Tuesday March 12, 2024 (2024/03/12) - Tuesday May 21, 2024 (2024/05/21)
PlayHQ Club fixture: https://bit.ly/bmbc-s2324


## 2. Get upcoming games for club's teams

First, get the teams of the club, sort them based on age group.

In [3]:
teams_df = phq_club.get_season_teams(SEASON_ID)
teams_df.sort_values('age', ascending=True, inplace=True)
teams_df.reset_index(inplace=True, drop=True)

teams = teams_df['name'].values
print(f"Found {len(teams)} teams:", teams)

# teams_df
# teams_df.query("name == 'Coburg U12 Boys 1'")

# if needed keep only a specific team (e.g., for debugging later)
# teams_df = teams_df.query("id == '115ca2a9-5383-4beb-9703-691006568974'")

Found 35 teams: ['Magic U10 Boys Black' 'Magic U10 Boys Gold' 'Magic U10 Boys Purple'
 'Magic U10 Girls Black' 'Magic U10 Girls Gold' 'Magic U10 Girls Purple'
 'Magic U12 Girls White' 'Magic U12 Girls Purple' 'Magic U12 Girls Gold'
 'Magic U12 Girls Diamond' 'Magic U12 Girls Black' 'Magic U12 Boys Purple'
 'Magic U12 Boys Gold' 'Magic U12 Boys Diamond' 'Magic U12 Boys Black'
 'Magic U12 Boys White' 'Magic U14 Girls Black' 'Magic U14 Girls Gold'
 'Magic U14 Girls Diamond' 'Magic U14 Boys White' 'Magic U14 Boys Diamond'
 'Magic U14 Boys Gold' 'Magic U14 Boys Black' 'Magic U14 Boys Purple'
 'Magic U16 Boys Black' 'Magic U16 Boys Diamond' 'Magic U16 Boys Gold'
 'Magic U16 Boys Purple' 'Magic U16 Girls Gold' 'Magic U16 Girls Purple'
 'Magic U18 Boys Gold' 'Magic U18 Boys Purple' 'Magic U18 Girls Gold'
 'Magic U8 Mixed Gold' 'Magic U8 Mixed Purple']


Next, extract all games between the dates specified for these teams of the club.

In [4]:
upcoming_games_df = phq_club.get_games(teams_df, GAME_DATE_START_TIMESTAMP, GAME_DATE_END_TIMESTAMP)

if upcoming_games_df is not None:
    print(f'There were {upcoming_games_df.shape[0]} games extracted for game between {GAME_DATE_START_NAME} and {GAME_DATE_END_NAME}')
    upcoming_games_df[phq.GAMES_COLS]
else:
    print(f'No games between {GAME_DATE_START_NAME} and {GAME_DATE_END_NAME}')

2024-03-13 13:20:47 INFO Games extracted for team: Magic U10 Boys Black
2024-03-13 13:20:47 INFO Games extracted for team: Magic U10 Boys Gold
2024-03-13 13:20:47 INFO Games extracted for team: Magic U10 Boys Purple
2024-03-13 13:20:47 INFO Games extracted for team: Magic U10 Girls Black
2024-03-13 13:20:47 INFO Games extracted for team: Magic U10 Girls Gold
2024-03-13 13:20:47 INFO Games extracted for team: Magic U10 Girls Purple
2024-03-13 13:20:47 INFO Games extracted for team: Magic U12 Girls White
2024-03-13 13:20:48 INFO No games for team: Magic U12 Girls Purple
2024-03-13 13:20:48 INFO No games for team: Magic U12 Girls Gold
2024-03-13 13:20:48 WARNING Games with ids ['4cc6a499-ac85-4b57-b64b-7e4895409319'] have no date. They will be dropped
2024-03-13 13:20:48 INFO No games for team: Magic U12 Girls Diamond
2024-03-13 13:20:48 WARNING Games with ids ['b7addf89-dcef-4fc0-b594-79619926d3c7'] have no date. They will be dropped
2024-03-13 13:20:48 INFO No games for team: Magic U12 

There were 16 games extracted for game between Tuesday March 12, 2024 (2024/03/12) and Tuesday May 21, 2024 (2024/05/21)


In [35]:
# dtale.show(upcoming_games_df)
upcoming_games_df

upcoming_games_df.query("team_name == 'U12 Boys Diamond'")
# upcoming_games_df.loc[upcoming_games_df.team_name == 'Magic U12 Girls White']

# upcoming_games_df.iloc[0,:]

,id,team_name,team_id,status,url,createdAt,updatedAt,pool,competitors,grade_id,...,venue_surfaceAbbreviation,venue_address_line1,venue_address_postcode,venue_address_suburb,venue_address_state,venue_address_country,venue_address_latitude,venue_address_longitude,venue,schedule_timestamp


## 3. Convert to TeamApp CSV format

Next, we convert the PlayHQ upcoming games to Teams App format so we can produce a CSV file to be imported into Teams App.

This process takes time as it processes game per game and even obtains short URL links for each game.

In [6]:
if upcoming_games_df is None:
    raise SystemExit("There are no games to process. Exiting.")

games_tapps_df = phq_club.to_teamsapp_schedule(upcoming_games_df, desc_template=DESC_TAPP, game_duration=45)
print("Done computing the games for Teams App")

# find out the game day if there is one day all teams play on only
game_day = None
single_game_day = (games_tapps_df['start_date'].drop_duplicates().size == 1)
if single_game_day:
    game_day = games_tapps_df.iloc[0]['start_date']
    print("All games are in the following day:", utils.pretty_date(game_day))

95710bba-cc2d-4ef3-beba-620ba422f88c [{'id': '95710bba-cc2d-4ef3-beba-620ba422f88c', 'name': 'Magic U10 Boys Black', 'isHomeTeam': False}, {'id': 'ac3e0fcc-87b7-499c-9062-d8e341df4b9a', 'name': 'Piranhas U10 Boys Teal', 'isHomeTeam': True}]
52e32a2a-e916-4c63-96fc-866b4e504f47 [{'id': '52e32a2a-e916-4c63-96fc-866b4e504f47', 'name': 'Magic U10 Boys Gold', 'isHomeTeam': True}, {'id': '6d6cf3cb-2e9b-4c19-9516-dddc8e7d5d40', 'name': 'Panthers U10 Boys Charcoal', 'isHomeTeam': False}]
b6342fe6-ca73-4771-a0ab-1478398983c6 [{'id': 'dfd0e085-d570-4c24-a099-46c94eb83edd', 'name': 'Warriors U10 Boys Red', 'isHomeTeam': False}, {'id': 'b6342fe6-ca73-4771-a0ab-1478398983c6', 'name': 'Magic U10 Boys Purple', 'isHomeTeam': True}]
a3ee2a00-0600-40f1-98a2-e1c3b2249f09 [{'id': 'a3ee2a00-0600-40f1-98a2-e1c3b2249f09', 'name': 'Magic U10 Girls Black', 'isHomeTeam': False}, {'id': 'e1cdae3b-7349-468b-aab1-c0a4c84d9c68', 'name': 'Piranhas U10 Girls Violet', 'isHomeTeam': True}]
b1ab7492-8268-4de3-848a-6844e

If in FINALS, there may be games scheduled for the following weekend, so they have no opponent yet.
We now list them to check and then drop them as they are not yet actual games.

In [14]:
games_tapps_df[games_tapps_df.opponent == "PENDING"]

,event_name,team_name,start_date,end_date,start_time,end_time,description,venue,location,access_groups,rsvp,comments,attendance_tracking,duty_roster,ticketing,opponent,court


In [13]:
games_tapps_df.drop(games_tapps_df[games_tapps_df.opponent == "PENDING"].index, inplace=True)

Inspect how the description of one of the games will look like:

In [8]:
# games_tapps_df.sample(3)
# dtale.show(games_tapps_df)
games_tapps_df

games_tapps_df.query("team_name == 'U12 Girls White'")

,event_name,team_name,start_date,end_date,start_time,end_time,description,venue,location,access_groups,rsvp,comments,attendance_tracking,duty_roster,ticketing,opponent,court
6,Game U12 Girls White - Grand Final,U12 Girls White,2024-03-23,2024-03-23,00:00:00,00:45:00,RSVP mandatory for the game.\n\nOpponent: PEND...,NaN,NaN,U12 Girls White,1,1,0,1,0,PENDING,NaN


Keep games after a particular date:

In [15]:
import datetime

START_DATE = None
# START_DATE = datetime.date(2023, 7, 21)

if START_DATE is not None:
    print("Keeping games after:", START_DATE)
    games_tapps_df[games_tapps_df['start_date'] > START_DATE]
    games_tapps_df = games_tapps_df[games_tapps_df['start_date'] > START_DATE]
else:
    print("Keeping all games")
games_tapps_df

Keeping all games


,event_name,team_name,start_date,end_date,start_time,end_time,description,venue,location,access_groups,rsvp,comments,attendance_tracking,duty_roster,ticketing,opponent,court
0,Game U10 Boys Black - Round 16,U10 Boys Black,2024-03-16,2024-03-16,09:30:00,10:15:00,RSVP mandatory for the game.\n\nOpponent: Pira...,Northcote High School,"19-25 St Georges Road, Northcote",U10 Boys Black,1,1,0,1,0,Piranhas U10 Boys Teal,Court 1
1,Game U10 Boys Gold - Round 16,U10 Boys Gold,2024-03-16,2024-03-16,10:30:00,11:15:00,RSVP mandatory for the game.\n\nOpponent: Pant...,Mercy College,"760 Sydney Road, Coburg",U10 Boys Gold,1,1,0,1,0,Panthers U10 Boys Charcoal,Court 1
2,Game U10 Boys Purple - Round 16,U10 Boys Purple,2024-03-16,2024-03-16,08:30:00,09:15:00,RSVP mandatory for the game.\n\nOpponent: Warr...,Northcote High School,"19-25 St Georges Road, Northcote",U10 Boys Purple,1,1,0,1,0,Warriors U10 Boys Red,Court 1
3,Game U10 Girls Black - Round 16,U10 Girls Black,2024-03-16,2024-03-16,10:30:00,11:15:00,RSVP mandatory for the game.\n\nOpponent: Pira...,Coburg Senior High School,"101 Urquhart Street, Coburg",U10 Girls Black,1,1,0,1,0,Piranhas U10 Girls Violet,Court 1
4,Game U10 Girls Gold - Round 16,U10 Girls Gold,2024-03-16,2024-03-16,09:30:00,10:15:00,RSVP mandatory for the game.\n\nOpponent: STAR...,Coburg Basketball Stadium,"25 Outlook Road, Coburg North",U10 Girls Gold,1,1,0,1,0,STARS U10 Girls SR,Court 4
5,Game U10 Girls Purple - Round 16,U10 Girls Purple,2024-03-16,2024-03-16,09:30:00,10:15:00,RSVP mandatory for the game.\n\nOpponent: Newl...,Coburg Senior High School,"101 Urquhart Street, Coburg",U10 Girls Purple,1,1,0,1,0,Newlands U10 Girls LEOS,Court 1
7,Game U12 Boys Gold - Preliminary Finals,U12 Boys Gold,2024-03-16,2024-03-16,11:30:00,12:15:00,RSVP mandatory for the game.\n\nOpponent: Jets...,Coburg Senior High School,"101 Urquhart Street, Coburg",U12 Boys Gold,1,1,0,1,0,Jets U12 Boys Yellow,Court 1
8,Game U12 Boys Black - Preliminary Final,U12 Boys Black,2024-03-16,2024-03-16,09:30:00,10:15:00,RSVP mandatory for the game.\n\nOpponent: St F...,Coburg Basketball Stadium,"25 Outlook Road, Coburg North",U12 Boys Black,1,1,0,1,0,St Fidelis U12 Boys Yellow,Court 1
10,Game U14 Boys Diamond - Finals Round 2,U14 Boys Diamond,2024-03-16,2024-03-16,11:30:00,12:15:00,RSVP mandatory for the game.\n\nOpponent: Rebe...,Coburg Basketball Stadium,"25 Outlook Road, Coburg North",U14 Boys Diamond,1,1,0,1,0,Rebels U14 Boys White,Court 2
12,Game U14 Boys Black - Finals Round 2,U14 Boys Black,2024-03-16,2024-03-16,12:30:00,13:15:00,RSVP mandatory for the game.\n\nOpponent: Jets...,Coburg Senior High School,"101 Urquhart Street, Coburg",U14 Boys Black,1,1,0,1,0,Jets U14 Boys Yellow,Court 1


In [19]:
# Inspect description game of one team
# team = "12.2 Boys"
team = "U14 Boys Black"

print("Description for:", team)
print(games_tapps_df.query("team_name == @team")['description'].values[0])

Description for: U14 Boys Black
RSVP mandatory for the game.

Opponent: Jets U14 Boys Yellow
Venue: Coburg Senior High School (Court 1)
Address: 101 Urquhart Street, Coburg 
Google Maps coord: https://maps.google.com/?q=(-37.74072,144.97181)
Waze coord: https://www.waze.com/live-map/directions?to=ll.-37.74072%2C144.97181

- Please ensure you arrive early and ready.
- Remember that shorts should have no pockets, players should not wear bracelets/watch as it is a risk of injury.
- No food in the venue and pickup your rubbish.
- Games will have 2x20 min halves.
- Each team needs to provide a scorer. TMs, please consider a roster.
- Players should not bring balls into the venue - game balls provided by Magic in coach's equipment bag.
- Beginners refs will be wearing green shirts. Please support and respect them through a POSITIVE sideline behaviour.

Check the game in PlayHQ: https://tinyurl.com/2cd6ux92
Check the round in PlayHQ: https://tinyurl.com/ypd7xl7z
Check all club's teams in Play

## 4. Append BYE games (if necessary)

We generate BYE entries for TeamsAPP ***only*** if the games are played all in the same day.

In [33]:
days_games = games_tapps_df['start_date'].drop_duplicates().values

if len(days_games) == 1:
    game_day = days_games[0]
    print("Seems all games are played on:", game_day)


Seems all games are played on: 2024-03-16


In [34]:
bye_teams= False    # assume no bye games

if game_day is not None:
    game_day = games_tapps_df.iloc[0]['start_date']

    # Extract the date of the round
    # date = games_tapps_df.iloc[1]['start_date']
    print(f"Extract BYE games for games on {utils.pretty_date(game_day)}")

    playing_teams = upcoming_games_df['team_id'].tolist()
    bye_teams = teams_df.loc[~teams_df['id'].isin(playing_teams)]['name'].tolist()
    bye_teams = list(map(lambda x: tapp_team_name(x), bye_teams))

    if bye_teams:
        games_bye_df = phq_club.build_teamsapp_bye_schedule(bye_teams, game_day, DESC_BYE_TAPP)
        print(f"Bye teams ({len(bye_teams)}): ", bye_teams)

        games_tapps_df = pd.concat([games_tapps_df, games_bye_df])
        games_tapps_df.drop_duplicates(inplace=True)
        games_tapps_df.reset_index(inplace=True, drop=True)
    else:
        print("No BYE games this round...")
else:
    print("Games obtained are not on the same day, not computing BYE games...")

(bye_teams and games_bye_df)

Extract BYE games for games on Saturday March 16, 2024 (2024/03/16)
Bye teams (19):  ['U12 Girls Purple', 'U12 Girls Gold', 'U12 Girls Diamond', 'U12 Girls Black', 'U12 Boys Purple', 'U12 Boys Diamond', 'U14 Girls Black', 'U14 Girls Gold', 'U14 Girls Diamond', 'U14 Boys White', 'U14 Boys Purple', 'U16 Boys Black', 'U16 Boys Gold', 'U16 Boys Purple', 'U16 Girls Purple', 'U18 Boys Gold', 'U18 Girls Gold', 'U08 Mixed Gold', 'U08 Mixed Purple']


,event_name,team_name,start_date,end_date,start_time,end_time,description,venue,location,access_groups,rsvp,comments,attendance_tracking,duty_roster,ticketing
0,U12 Girls Purple - BYE,U12 Girls Purple,2024-03-16,2024-03-16,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U12 Girls Purple,0,0,0,0,0
1,U12 Girls Gold - BYE,U12 Girls Gold,2024-03-16,2024-03-16,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U12 Girls Gold,0,0,0,0,0
2,U12 Girls Diamond - BYE,U12 Girls Diamond,2024-03-16,2024-03-16,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U12 Girls Diamond,0,0,0,0,0
3,U12 Girls Black - BYE,U12 Girls Black,2024-03-16,2024-03-16,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U12 Girls Black,0,0,0,0,0
4,U12 Boys Purple - BYE,U12 Boys Purple,2024-03-16,2024-03-16,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U12 Boys Purple,0,0,0,0,0
5,U12 Boys Diamond - BYE,U12 Boys Diamond,2024-03-16,2024-03-16,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U12 Boys Diamond,0,0,0,0,0
6,U14 Girls Black - BYE,U14 Girls Black,2024-03-16,2024-03-16,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U14 Girls Black,0,0,0,0,0
7,U14 Girls Gold - BYE,U14 Girls Gold,2024-03-16,2024-03-16,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U14 Girls Gold,0,0,0,0,0
8,U14 Girls Diamond - BYE,U14 Girls Diamond,2024-03-16,2024-03-16,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U14 Girls Diamond,0,0,0,0,0
9,U14 Boys White - BYE,U14 Boys White,2024-03-16,2024-03-16,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U14 Boys White,0,0,0,0,0


## 5. Final review

Finally, we report the games to be written into Schedule CSV file and **CHECKING THAT ALL IS GOOD TO GO!**

Particularly, look for games that are schedule but **PENDING** and without all details (time or venue).

In [36]:
games_tapps_df.columns
games_tapps_df[['event_name', 'team_name', 'opponent', 'start_date', 'start_time', 'venue', 'court']]
# games_tapps_df

,event_name,team_name,opponent,start_date,start_time,venue,court
0,Game U10 Boys Black - Round 16,U10 Boys Black,Piranhas U10 Boys Teal,2024-03-16,09:30:00,Northcote High School,Court 1
1,Game U10 Boys Gold - Round 16,U10 Boys Gold,Panthers U10 Boys Charcoal,2024-03-16,10:30:00,Mercy College,Court 1
2,Game U10 Boys Purple - Round 16,U10 Boys Purple,Warriors U10 Boys Red,2024-03-16,08:30:00,Northcote High School,Court 1
3,Game U10 Girls Black - Round 16,U10 Girls Black,Piranhas U10 Girls Violet,2024-03-16,10:30:00,Coburg Senior High School,Court 1
4,Game U10 Girls Gold - Round 16,U10 Girls Gold,STARS U10 Girls SR,2024-03-16,09:30:00,Coburg Basketball Stadium,Court 4
5,Game U10 Girls Purple - Round 16,U10 Girls Purple,Newlands U10 Girls LEOS,2024-03-16,09:30:00,Coburg Senior High School,Court 1
6,Game U12 Boys Gold - Preliminary Finals,U12 Boys Gold,Jets U12 Boys Yellow,2024-03-16,11:30:00,Coburg Senior High School,Court 1
7,Game U12 Boys Black - Preliminary Final,U12 Boys Black,St Fidelis U12 Boys Yellow,2024-03-16,09:30:00,Coburg Basketball Stadium,Court 1
8,Game U14 Boys Diamond - Finals Round 2,U14 Boys Diamond,Rebels U14 Boys White,2024-03-16,11:30:00,Coburg Basketball Stadium,Court 2
9,Game U14 Boys Black - Finals Round 2,U14 Boys Black,Jets U14 Boys Yellow,2024-03-16,12:30:00,Coburg Senior High School,Court 1


We stop the execution here if we are running all Jupyter notebook.

In [ ]:
raise SystemExit("Stop right there! Continue below to produce the CSV file if needed.")

## 6. Save to CSV file for Teams App import

OK we are ready to import into Teams App.

### 6.2. Check changes with previous saves

If the schedule was generated before, check if the new one differs with the one saved already.

First, let us define the files that we will save to disk.

In [11]:
now = datetime.datetime.now() # current date and time
now_str = now.strftime("%Y_%m_%d-%H:%M:%S")

id_file = now_str
if game_day is not None:    # there is one date for all games!
    id_file = utils.compact_date(game_day)

file_csv = os.path.join(OUTPUT_PATH, f"schedule-teamsapp-{id_file}.csv")
file_upcoming_pkl = os.path.join(OUTPUT_PATH, f"upcoming_games_df-{id_file}.pkl")
file_games_tapps = os.path.join(OUTPUT_PATH, f"games_tapps_df-{id_file}.pkl")

print("Files to save:")
print(file_csv)
print(file_upcoming_pkl)
print(file_games_tapps)

if not os.path.exists(OUTPUT_PATH):
    raise SystemExit("ERROR! Output path {OUTPUT_PATH} is missing! Please create or link that path correctly to save data.")

Files to save:
Coburg_Rep/2023-24/schedule-teamsapp-2024_03_12-19:32:02.csv
Coburg_Rep/2023-24/upcoming_games_df-2024_03_12-19:32:02.pkl
Coburg_Rep/2023-24/games_tapps_df-2024_03_12-19:32:02.pkl


Next, let's check if there was a saved file for the upcoming game day.

In [12]:
cols = ['team_name', 'opponent', 'start_date', 'start_time', 'venue', 'court']

changed_games_df = None
if os.path.exists(file_games_tapps):
    print("There was already a schedule saved, recovering it to compare...")
    old_games_tapps_df = pd.read_pickle(file_games_tapps)

    teams_changed = pd.concat([games_tapps_df[cols], old_games_tapps_df[cols]]).drop_duplicates(keep=False)['team_name'].unique()
    print("Teams whose games have changed (updated, new, dropped):", teams_changed)

    old_games_df = old_games_tapps_df[cols].query("team_name in @teams_changed")
    new_games_df = games_tapps_df[cols].query("team_name in @teams_changed")
    changed_games_df = new_games_df.merge(old_games_df, how="inner", on="team_name", suffixes=('_new', '_old'))
else:
    print("No previous schedule saved")

# Show changes if any...
changed_games_df

No previous schedule saved


### 5.3. Write a TeamAPP Schedule CSV & Datafarmes Pickles

Finally, we save the data to a CSV file that can be imported into the [SCHEDULE of TeamsApp for all Entries](https://brunswickmagicbasketball.teamapp.com/clubs/263995/events?_list=v1&team_id=all).

In [13]:
import shutil

print('Saving TeamAPP schedule CSV file and Dataframes with id:', id_file)
for f in [file_csv, file_upcoming_pkl, file_games_tapps]:
  if os.path.exists(f):
    print("Backup file", f)
    shutil.copy(f, f + ".bak")

print('Saving CSV TeamApp schedule:', file_csv)
games_tapps_df.to_csv(file_csv, index=False)

print('Saving dataframe pickle:', file_upcoming_pkl)
upcoming_games_df.to_pickle(file_upcoming_pkl)
print('Saving dataframe pickle:', file_games_tapps)
games_tapps_df.to_pickle(file_games_tapps)

print(f"Finished saving CSV and DATA-FRAMNE files: {now.strftime('%d/%m/%Y, %H:%M:%S')}")

Saving TeamAPP schedule CSV file and Dataframes with id: 2024_03_12-19:32:02
Saving CSV TeamApp schedule: Coburg_Rep/2023-24/schedule-teamsapp-2024_03_12-19:32:02.csv
Saving dataframe pickle: Coburg_Rep/2023-24/upcoming_games_df-2024_03_12-19:32:02.pkl
Saving dataframe pickle: Coburg_Rep/2023-24/games_tapps_df-2024_03_12-19:32:02.pkl
Finished saving CSV and DATA-FRAMNE files: 12/03/2024, 19:32:02


# ------------ END FIXTURE PUBLISHING ------------

### Check a particular team

In [ ]:
team = "U10 Girls Gold"

print(games_tapps_df.query("team_name == @team")['description'].values[0])
games_tapps_df.query("team_name == @team")[['team_name', 'opponent', 'start_date', 'start_time', 'venue', 'court']]
